In [56]:
from sklearn.datasets import fetch_openml
import numpy as np
import pandas as pd
import cv2

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

In [2]:
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
mnist.keys()

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [3]:
x, y = mnist['data'], mnist['target']
x.shape, y.shape

((70000, 784), (70000,))

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, shuffle=True)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((49000, 784), (21000, 784), (49000,), (21000,))

In [5]:
knn_clf = KNeighborsClassifier()
knn_clf.fit(x_train, y_train)

KNeighborsClassifier()

In [ ]:
y_pred_train = knn_clf.predict(x_train)
y_pred_test = knn_clf.predict(x_test)

print(f'Training accuracy: {accuracy_score(y_train, y_pred_train)}:.2f')
print(f'Testing accuracy: {accuracy_score(y_test, y_pred_test)}:.2f')


Training accuracy: 0.98
Testing accuracy: 0.97


In [7]:
# hyperparameter tuning
param_grid = [{
    'weights': ['uniform', 'distance'],
    'n_neighbors': [3,4,5,6,7]
}]

rndm_srch_cv = RandomizedSearchCV(knn_clf, param_grid, cv=5, verbose=3, n_jobs=3, scoring='accuracy')
rndm_srch_cv.fit(x_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 2/5] END ....n_neighbors=3, weights=uniform;, score=0.971 total time=  38.8s
[CV 3/5] END ....n_neighbors=3, weights=uniform;, score=0.967 total time=  38.9s
[CV 1/5] END ....n_neighbors=3, weights=uniform;, score=0.971 total time=  39.0s
[CV 1/5] END ...n_neighbors=3, weights=distance;, score=0.971 total time=  32.9s
[CV 5/5] END ....n_neighbors=3, weights=uniform;, score=0.970 total time=  32.9s
[CV 4/5] END ....n_neighbors=3, weights=uniform;, score=0.967 total time=  33.2s
[CV 3/5] END ...n_neighbors=3, weights=distance;, score=0.968 total time=  25.8s
[CV 2/5] END ...n_neighbors=3, weights=distance;, score=0.972 total time=  26.2s
[CV 4/5] END ...n_neighbors=3, weights=distance;, score=0.968 total time=  26.1s
[CV 2/5] END ....n_neighbors=4, weights=uniform;, score=0.968 total time=  27.3s
[CV 1/5] END ....n_neighbors=4, weights=uniform;, score=0.969 total time=  27.6s
[CV 5/5] END ...n_neighbors=3, weights=distance;

RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=3,
                   param_distributions=[{'n_neighbors': [3, 4, 5, 6, 7],
                                         'weights': ['uniform', 'distance']}],
                   scoring='accuracy', verbose=3)

In [8]:
rndm_srch_cv.best_score_

0.9713265306122449

In [9]:
y_pred_test_hyp = rndm_srch_cv.best_estimator_.predict(x_test)
accuracy_score(y_test, y_pred_test_hyp)

0.9713809523809523

### exercise 2

In [17]:
def img_shifting(img):
    img = img.reshape((28,28))
    img = img.astype(np.uint8)
    rows, cols = img.shape[:2]
    m_left = np.float32([[1,0,-1], [0,1,0]])
    m_right = np.float32([[1,0,1], [0,1,0]])
    m_up = np.float32([[1,0,0], [0,1,-1]])
    m_down = np.float32([[1,0,0], [0,1,1]])

    shift_left = cv2.warpAffine(img, m_left, (cols, rows), borderValue=0)
    shift_right = cv2.warpAffine(img, m_right, (cols, rows), borderValue=0)
    shift_up = cv2.warpAffine(img, m_up, (cols, rows), borderValue=0)
    shfit_down = cv2.warpAffine(img, m_down, (cols, rows), borderValue=0)

    return [shift_left, shift_right, shift_up, shfit_down]


In [18]:
x_aug, y_aug = [], []

for img, label in zip(x_train, y_train):
    shifted_imgs = img_shifting(img)
    x_aug.extend(shifted_imgs)
    y_aug.extend([label] * len(shifted_imgs))

x_aug = np.array(x_aug)
y_aug = np.array(y_aug)

In [20]:
x_aug.shape, y_aug.shape

((196000, 28, 28), (196000,))

In [24]:
x_train.shape

(49000, 784)

In [28]:
x_aug_flat = x_aug.reshape((x_aug.shape[0], -1))

x_train_full = np.concatenate([x_train, x_aug_flat], axis=0)
y_train_full = np.concatenate([y_train, y_aug], axis=0)

In [29]:
rndm_srch_cv.best_estimator_.fit(x_train_full, y_train_full)
y_pred_full = rndm_srch_cv.best_estimator_.predict(x_test)

In [30]:
accuracy_score(y_test, y_pred_full)

0.9784285714285714

## exercise 3

In [101]:
# titanic dataset to predict if passenger survived or not (binary classifier)
df_train = pd.read_csv('../datasets/train_titanic.csv')
df_test = pd.read_csv('../datasets/test_titanic.csv')
submission = pd.read_csv('../datasets/gender_submission.csv')
df_train.shape, df_test.shape, submission.shape

((891, 12), (418, 11), (418, 2))

In [102]:
df_train.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [103]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [104]:
df_train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [105]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [106]:
df_train['Sex'].value_counts()

Sex
male      577
female    314
Name: count, dtype: int64

In [107]:
df_train['Age'].min(), df_train['Age'].max(), df_train['Age'].mean()

(0.42, 80.0, 29.69911764705882)

In [108]:
df_train['Sex'] = df_train['Sex'].replace({'female': 0, 'male': 1})
df_test['Sex'] = df_test['Sex'].replace({'female': 0, 'male': 1})

In [109]:
df_train['Age'] = df_train['Age'].fillna(df_train['Age'].mean())

In [110]:
df_test.head(2)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",1,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",0,47.0,1,0,363272,7.0000,NaN,S


In [111]:
df_train.Cabin.value_counts()

Cabin
B96 B98        4
G6             4
C23 C25 C27    4
C22 C26        3
F33            3
              ..
E34            1
C7             1
C54            1
E36            1
C148           1
Name: count, Length: 147, dtype: int64

In [112]:
df_train['Survived'].value_counts()

Survived
0    549
1    342
Name: count, dtype: int64

In [113]:
x = df_train.drop(['Cabin', 'Embarked', 'PassengerId', 'Ticket', 'Name', 'Fare','Survived'], axis=1)
test = df_test.drop(['Cabin', 'Embarked', 'PassengerId', 'Ticket', 'Name', 'Fare'],axis=1)
y = df_train['Survived'].values

In [114]:
x.shape, y.shape

((891, 5), (891,))

In [115]:
x.head()

,Pclass,Sex,Age,SibSp,Parch
0,3,1,22.0,1,0
1,1,0,38.0,1,0
2,3,0,26.0,0,0
3,1,0,35.0,1,0
4,3,1,35.0,0,0


In [116]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle=True)
x_train.shape, x_test.shape, y_train.shape, y_test.shape, df_test.shape

((712, 5), (179, 5), (712,), (179,), (418, 11))

### scaling columns

In [117]:
scaler = StandardScaler()
scaler.fit(x_train, y_train)

StandardScaler()

In [118]:
x_train

,Pclass,Sex,Age,SibSp,Parch
741,1,1,36.000000,1,0
176,3,1,29.699118,3,1
652,3,1,21.000000,0,0
470,3,1,29.699118,0,0
804,3,1,27.000000,0,0
...,...,...,...,...,...
511,3,1,29.699118,0,0
742,1,0,21.000000,2,2
767,3,0,30.500000,0,0
314,2,1,43.000000,1,1


In [125]:
def modelling(model, param='train', x_train=x_train, y_train=y_train, x_test=x_test, y_test=y_test):
    model.fit(x_train, y_train)

    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)
    if param == 'train':
        acc_score_train = accuracy_score(y_train, y_pred_train)
        acc_score_test = accuracy_score(y_test, y_pred_test)
        f1_score_train = f1_score(y_train, y_pred_train)
        f1_score_test = f1_score(y_test, y_pred_test)
        return acc_score_train, acc_score_test, f1_score_train, f1_score_test
    elif param == 'predict' and submission is not None:
        sub = model.predict(x_test)
        submission['Survived'] = sub
        model_name = type(model).__name__
        submission.to_csv(f'../datasets/'+'{model_name}_submission.csv', index=False)
        return submission
    else:
        raise ValueError("Invalid parameter: provide 'predict' with a submission_df")

    

In [126]:
log_reg = LogisticRegression()
knn_clf_1 = KNeighborsClassifier()
rnd_frst = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)

mdls_lst = [log_reg, knn_clf_1, rnd_frst]

In [127]:
for model in mdls_lst:
    acc_score_train, acc_score_test, f1_score_train, f1_score_test = modelling(model, 'train', x_train, y_train, x_test, y_test)
    print(str(model))
    print(f'Accuracy Score: {acc_score_train:.2f}')
    print(f'Accuracy Score Test: {acc_score_test:.2f}')
    print(f'F1 Score: {f1_score_train:.2f}')
    print(f'F1 Score Test: {f1_score_test:.2f}')
    print('*' * 20)

LogisticRegression()
Accuracy Score: 0.80
Accuracy Score Test: 0.79
F1 Score: 0.75
F1 Score Test: 0.70
********************
KNeighborsClassifier()
Accuracy Score: 0.83
Accuracy Score Test: 0.83
F1 Score: 0.78
F1 Score Test: 0.75
********************
RandomForestClassifier(max_depth=5, random_state=1)
Accuracy Score: 0.85
Accuracy Score Test: 0.83
F1 Score: 0.79
F1 Score Test: 0.74
********************


In [128]:
modelling(rnd_frst,'predict', x_train, y_train, test, y_test)

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
